In [2]:
import folium
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
data_path = 'Data/AFAD_1900_today_31461754_4_9.csv'
df = pd.read_csv(data_path)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y %H:%M:%S')

In [4]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour
df['Previous_Magnitude'] = df['Magnitude'].shift(1).bfill()

In [5]:
features = ['Longitude', 'Latitude', 'Depth', 'Year', 'Month', 'Day', 'Hour', 'Previous_Magnitude']
X = df[features]
y = df['Magnitude']
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
df.head()

,Date,Longitude,Latitude,Depth,Type,Magnitude,Location,EventID,TimeName,TypeName,MagnitudeName,Year,Month,Day,Hour,Previous_Magnitude
0,1903-04-29 16:07:56,42.65,39.15,33.0,MS,5.0,-,7821,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,1903,4,29,16,5.0
1,1903-04-30 04:12:56,42.65,39.15,10.0,MS,4.6,-,7822,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,1903,4,30,4,5.0
2,1903-05-03 01:56:56,41.50,38.70,10.0,MS,4.9,-,7823,Ayhan ve Diğ.1981,Ayhan ve Diğ.1981,Ayhan ve Diğ.1981,1903,5,3,1,4.6
3,1903-05-21 20:56:56,42.50,39.00,10.0,MS,4.2,-,7824,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,1903,5,21,20,4.9
4,1903-05-26 08:05:56,29.00,40.65,10.0,MS,5.9,-,7825,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,1903,5,26,8,4.2


In [7]:
# Model
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
error = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {error}')

Mean Squared Error: 0.14707262943372323


In [8]:
next_eq_features = pd.DataFrame([[37.84, 32.36, 10.0, 2023, 1, 1, 12, 4.5]], columns=features)
predicted_magnitude = model.predict(next_eq_features)
print(f'Predicted Magnitude of the next earthquake: {predicted_magnitude[0]}')

Predicted Magnitude of the next earthquake: 4.659999999999999


# Map visualization

In [13]:
map = folium.Map(location=[38.9637, 35.2433], zoom_start=5)  # Centered on Turkey

# Add actual earthquake locations
for idx, row in X_test.iterrows():
    folium.Circle(
        location=[row['Latitude'], row['Longitude']],
        radius=2** y_test.loc[idx],  # Changed from iloc to loc
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        popup=f"Actual Magnitude: {y_test.loc[idx]}"
    ).add_to(map)
map.save('earthquake_real.html')
# Add predicted earthquake locations
for idx, (index, row) in enumerate(X_test.iterrows()):
    folium.Circle(
        location=[row['Latitude'], row['Longitude']],
        radius=2** y_pred[idx],  # Radius scaled by predicted magnitude
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        popup=f"Predicted Magnitude: {y_pred[idx]}"
    ).add_to(map)

# Save or show the map
map.save('earthquake_predictions.html')

# Grid predictions

In [10]:
# Define the bounds of the grid
lat_start, lat_end = 36, 42  # Rough latitude bounds for Turkey
lon_start, lon_end = 26, 45  # Rough longitude bounds for Turkey

# Generate grid points with 10km spacing
# Assuming roughly 111 km per degree of latitude or longitude
km_per_degree = 111
spacing = 10 / km_per_degree  # Convert 10 km into degrees

latitudes = np.arange(lat_start, lat_end, spacing)
longitudes = np.arange(lon_start, lon_end, spacing)

In [11]:
mean_previous_magnitude = df['Magnitude'].mean()  # Calculate the mean of magnitudes

# Generate grid points
mean_depth = df['Depth'].mean()
grid_points = [(lon, lat, mean_depth, 2020, 1, 1, 12, mean_previous_magnitude) 
               for lat in latitudes for lon in longitudes]

# Create DataFrame for predictions
grid_df = pd.DataFrame(grid_points, columns=['Longitude','Latitude',  'Depth', 
                                             'Year', 'Month', 'Day', 'Hour', 'Previous_Magnitude'])

# Predict magnitudes
predicted_magnitudes = model.predict(grid_df)


In [12]:
lat_start, lat_end = 36, 42  # Rough latitude bounds for Turkey
lon_start, lon_end = 26, 45  # Rough longitude bounds for Turkey
map = folium.Map(location=[(lat_start + lat_end) / 2, (lon_start + lon_end) / 2], zoom_start=6)

# Assuming predicted_magnitudes corresponds to each point in grid_points
for point, mag in zip(grid_points, predicted_magnitudes):
    lon, lat = point[0], point[1]  # Corrected unpacking
    # Convert magnitude to color, assuming a range of magnitudes from 3 to 7 for example
    norm_mag = (mag - 3) / (7 - 3)  # Normalize magnitude to range [0,1]
    color = plt.cm.RdBu_r(norm_mag)  # Red to Blue color map, high magnitude = red
    
    # Add circle
    folium.Circle(
        location=[lat, lon],
        radius=2000,  # Example fixed radius, scale as needed
        color=plt.colors.to_hex(color),
        fill=True,
        fill_color=plt.colors.to_hex(color),
        fill_opacity=0.7,
        popup=f"Predicted Magnitude: {mag:.2f}"
    ).add_to(map)

# Save or display the map
map.save('earthquake_grid_predictions.html')